In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SLDE /GEEAQCSABLKNTTOGAA)RLS ( NM', 'calculationPrice': 'close', 'open': 137.81, 'openTime': 1651563260532, 'openSource': 'fliifaco', 'close': 140.47, 'closeTime': 1620347438193, 'closeSource': 'oiiclffa', 'high': 139.68, 'highTime': 1619678180873, 'highSource': 'TILtX aadeE rs', 'low': 134.95, 'lowTime': 1687808407880, 'lowSource': 'e e5ltncddra iypie1 mue', 'latestPrice': 141.41, 'latestSource': 'Close', 'latestTime': 'February 12, 2021', 'latestUpdate': 1675694401715, 'latestVolume': 61503629, 'iexRealtimePrice': 137.9, 'iexRealtimeSize': 1, 'iexLastUpdated': 1619532858698, 'delayedPrice': 139.26, 'delayedPriceTime': 1645766868073, 'oddLotDelayedPrice': 141.7, 'oddLotDelayedPriceTime': 1632759657319, 'extendedPrice': 140.34, 'extendedChange': 0.06, 'extendedChangePercent': 0.00045, 'extendedPriceTime': 1628272322087, 'previousClose': 137.83, 'previousVolume': 65000772, 'change': 0.25, 'changePercent': 0.00187, 'volum

In [7]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

38.06

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data.status_code)
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
        )
        
#        print(symbol)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,129.44,58.06,N/A
1,AAL,17.57,-1.27,N/A
2,AAP,155.62,23.32,N/A
3,AAPL,138.02,37.69,N/A
4,ABBV,107.96,23.8,N/A
...,...,...,...,...
500,YUM,108.28,31.51,N/A
501,ZBH,165.76,-227.19,N/A
502,ZBRA,489.15,54.64,N/A
503,ZION,52.01,21.8,N/A
